In [3]:
import re

In [4]:
with open('./Dump20200110.sql', mode='r', encoding='utf-8') as f:
    txt = f.read()

In [5]:
{'databases':[{'name':'db1', 'tables':[{'name':'tb11'}]}, {'name':'db2', 'tables':[{'name':'tb21'}]}]}

{'databases': [{'name': 'db1', 'tables': [{'name': 'tb11'}]},
  {'name': 'db2', 'tables': [{'name': 'tb21'}]}]}

In [6]:
print(sql_create_tables)

NameError: name 'sql_create_tables' is not defined

In [7]:
sql

NameError: name 'sql' is not defined

In [136]:
mysql = {'databases':[]}
schemas_re = list(re.finditer(r'CREATE DATABASE.+`(.+)`', txt, re.MULTILINE))
#txt[schemas_re[0].end():]
for i in range(len(schemas_re)):
    database_name = schemas_re[i].groups()[0]
    database = {'name':database_name, 'tables':[]}
    start_sql_db = schemas_re[i].end()
    j = i+1
    if j != len(schemas_re): 
        end_sql_db = schemas_re[j].start()
    else:
        end_sql_db = len(txt)
    sql_create_tables = txt[start_sql_db:end_sql_db]
    
    tables_re = list(re.finditer(r'CREATE TABLE.+`(.+)`', sql_create_tables, re.MULTILINE))
    for k in range(len(tables_re)):
        table_name = tables_re[k].groups()[0]
        table = {'name':table_name, 'columns':[]}
        start_sql_table = tables_re[k].end()
        l = k+1
        if l != len(tables_re): 
            end_sql_table = tables_re[l].start()
        else:
            end_sql_table = len(sql_create_tables)
        sql_columns = sql_create_tables[start_sql_table:end_sql_table]
        
        # TODO: identify composed PK
        # TODO: identify FK table and FK column.
        pk_re = re.search(r'PRIMARY KEY \(`(\w+)`\)', sql_columns)
        if pk_re: 
            column_pk = pk_re.groups()[0]
            
            
        columns_re = list(re.finditer(r'`(.*?)`\s(.+?)\s', sql_columns, re.MULTILINE))        
        for m in range(len(columns_re)):
            column_name = columns_re[m].group(1)
            column_type = columns_re[m].group(2)
            column = {'name':column_name, 'type':column_type}
            if column_name == column_pk: column['primary-key']=True
            table['columns'].append(column)
        database['tables'].append(table)
    mysql['databases'].append(database)
